In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("f1-embed-lapdata.csv", low_memory=False)

In [3]:
df.head()

,Date,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Time_x,SessionTime,...,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Track
0,2022-07-01 12:02:53.928,10687.0,232.0,6,99.0,False,9,car,0 days 00:00:00.010000,0 days 00:17:53.920000,...,Ferrari,0 days 00:17:53.910000,2022-07-01 12:02:53.918,1,NaN,False,NaN,False,True,british grand prix
1,2022-07-01 12:02:54.088,10796.0,236.0,6,99.0,False,9,car,0 days 00:00:00.170000,0 days 00:17:54.080000,...,Ferrari,0 days 00:17:53.910000,2022-07-01 12:02:53.918,1,NaN,False,NaN,False,True,british grand prix
2,2022-07-01 12:02:54.448,11077.0,243.0,6,99.0,False,9,car,0 days 00:00:00.530000,0 days 00:17:54.440000,...,Ferrari,0 days 00:17:53.910000,2022-07-01 12:02:53.918,1,NaN,False,NaN,False,True,british grand prix
3,2022-07-01 12:02:54.768,11288.0,248.0,6,100.0,False,9,car,0 days 00:00:00.850000,0 days 00:17:54.760000,...,Ferrari,0 days 00:17:53.910000,2022-07-01 12:02:53.918,1,NaN,False,NaN,False,True,british grand prix
4,2022-07-01 12:02:55.008,11473.0,251.0,6,100.0,False,9,car,0 days 00:00:01.090000,0 days 00:17:55,...,Ferrari,0 days 00:17:53.910000,2022-07-01 12:02:53.918,1,NaN,False,NaN,False,True,british grand prix


In [4]:
df.columns

Index(['Date', 'RPM', 'Speed', 'nGear', 'Throttle', 'Brake', 'DRS', 'Source',
       'Time_x', 'SessionTime', 'Distance', 'Driver', 'LapNumber', 'Year',
       'Round', 'SessionType', 'Time_y', 'DriverNumber', 'LapTime', 'Stint',
       'PitOutTime', 'PitInTime', 'Sector1Time', 'Sector2Time', 'Sector3Time',
       'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime',
       'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 'IsPersonalBest',
       'Compound', 'TyreLife', 'FreshTyre', 'Team', 'LapStartTime',
       'LapStartDate', 'TrackStatus', 'Position', 'Deleted', 'DeletedReason',
       'FastF1Generated', 'IsAccurate', 'Track'],
      dtype='object')

In [5]:
df.columns[df.isna().any()].tolist()

['PitOutTime',
 'PitInTime',
 'Sector1SessionTime',
 'SpeedI1',
 'SpeedST',
 'Position',
 'DeletedReason']

In [6]:
req_columns = ["RPM", "Speed", "nGear", "Throttle", "Brake", "DRS", "Time_x", "Driver", "LapNumber", "Year", "Round", "Compound", "TyreLife", "FreshTyre", "Team", "LapTime", "TrackStatus",]

In [7]:
df = df[req_columns]

In [8]:
df.columns

Index(['RPM', 'Speed', 'nGear', 'Throttle', 'Brake', 'DRS', 'Time_x', 'Driver',
       'LapNumber', 'Year', 'Round', 'Compound', 'TyreLife', 'FreshTyre',
       'Team', 'LapTime', 'TrackStatus'],
      dtype='object')

In [9]:
input_features = ["RPM", "Speed", "nGear", "Throttle", "Brake", "DRS", "Time_x"]
context_features = ["Driver", "LapNumber", "Year", "Round", "Compound", "TyreLife", "FreshTyre", "Team", "TrackStatus"]
target = "LapTime"

In [10]:
laps = df.groupby(["Driver", "LapNumber", "Year", "Round"])

In [11]:
laps.head()

,RPM,Speed,nGear,Throttle,Brake,DRS,Time_x,Driver,LapNumber,Year,Round,Compound,TyreLife,FreshTyre,Team,LapTime,TrackStatus
0,10687.0,232.0,6,99.0,False,9,0 days 00:00:00.010000,LEC,2.0,2022,silverstone,INTERMEDIATE,2.0,True,Ferrari,0 days 00:01:44.769000,1
1,10796.0,236.0,6,99.0,False,9,0 days 00:00:00.170000,LEC,2.0,2022,silverstone,INTERMEDIATE,2.0,True,Ferrari,0 days 00:01:44.769000,1
2,11077.0,243.0,6,99.0,False,9,0 days 00:00:00.530000,LEC,2.0,2022,silverstone,INTERMEDIATE,2.0,True,Ferrari,0 days 00:01:44.769000,1
3,11288.0,248.0,6,100.0,False,9,0 days 00:00:00.850000,LEC,2.0,2022,silverstone,INTERMEDIATE,2.0,True,Ferrari,0 days 00:01:44.769000,1
4,11473.0,251.0,6,100.0,False,9,0 days 00:00:01.090000,LEC,2.0,2022,silverstone,INTERMEDIATE,2.0,True,Ferrari,0 days 00:01:44.769000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847248,10883.0,270.0,7,99.0,False,0,0 days 00:00:00.163000,SAI,53.0,2024,suzuka,HARD,17.0,True,Ferrari,0 days 00:01:35.224000,1
847249,10964.0,272.0,7,99.0,False,0,0 days 00:00:00.323000,SAI,53.0,2024,suzuka,HARD,17.0,True,Ferrari,0 days 00:01:35.224000,1
847250,11097.0,275.0,7,99.0,False,0,0 days 00:00:00.603000,SAI,53.0,2024,suzuka,HARD,17.0,True,Ferrari,0 days 00:01:35.224000,1
847251,11160.0,280.0,7,99.0,False,0,0 days 00:00:00.963000,SAI,53.0,2024,suzuka,HARD,17.0,True,Ferrari,0 days 00:01:35.224000,1


In [12]:
laps.sum().shape

(1770, 13)

In [13]:
for lap_key, lap_df in laps:
    driver, lap, year, round_ = lap_key

    lap_time = lap_df["LapTime"].iloc[0]
    compound = lap_df["Compound"].iloc[0]
    team = lap_df["Team"].iloc[0]
    rows = len(lap_df)

    print(f"Lap Name: {lap_key} | Driver: {driver} | Lap Time: {lap_time} | Team: {team} | Compound: {compound}")

Lap Name: ('LEC', np.float64(2.0), np.int64(2022), 'monaco') | Driver: LEC | Lap Time: 0 days 00:01:14.892000 | Team: Ferrari | Compound: SOFT
Lap Name: ('LEC', np.float64(2.0), np.int64(2022), 'silverstone') | Driver: LEC | Lap Time: 0 days 00:01:44.769000 | Team: Ferrari | Compound: INTERMEDIATE
Lap Name: ('LEC', np.float64(2.0), np.int64(2022), 'spa') | Driver: LEC | Lap Time: 0 days 00:01:23.886000 | Team: Ferrari | Compound: HARD
Lap Name: ('LEC', np.float64(2.0), np.int64(2022), 'suzuka') | Driver: LEC | Lap Time: 0 days 00:01:49.103000 | Team: Ferrari | Compound: WET
Lap Name: ('LEC', np.float64(2.0), np.int64(2023), 'monaco') | Driver: LEC | Lap Time: 0 days 00:01:17.131000 | Team: Ferrari | Compound: HARD
Lap Name: ('LEC', np.float64(2.0), np.int64(2023), 'silverstone') | Driver: LEC | Lap Time: 0 days 00:01:31.663000 | Team: Ferrari | Compound: MEDIUM
Lap Name: ('LEC', np.float64(2.0), np.int64(2023), 'spa') | Driver: LEC | Lap Time: 0 days 00:01:15.600000 | Team: Ferrari | C

In [14]:
def resample_lap_np_interp(lap_df, input_features, target_len=100):
    if not np.issubdtype(lap_df["Time_x"].dtype, np.number):
        lap_df = lap_df.copy()
        lap_df["Time_x"] = lap_df["Time_x"].dt.total_seconds()

    time = lap_df["Time_x"].values
    if len(np.unique(time)) < 2:
        raise ValueError("Not enough unique time points to interpolate.")

    # New uniform time axis
    time_resampled = np.linspace(time[0], time[-1], num=target_len)

    resampled = []
    for col in input_features:
        if col == "Time_x":
            continue
        y = lap_df[col].values

        if y.dtype == bool:
            y = y.astype(float)

        y_interp = np.interp(time_resampled, time, y)
        resampled.append(y_interp)

    return np.stack(resampled, axis=1)

In [15]:
from tqdm import tqdm

X_seq = []
X_context = []
y = []

for lap_key, lap_df in tqdm(laps, desc="Processing Laps"):
    try:
        lap_data = lap_df[input_features].copy()
        lap_data["Time_x"] = pd.to_timedelta(lap_data["Time_x"], errors="coerce")
        lap_data["Time_x"] = lap_data["Time_x"].dt.total_seconds()
        
        if len(lap_data) < 10:
            continue
        
        resampled = resample_lap_np_interp(lap_data, input_features)
        
        X_seq.append(resampled)
        X_context.append(lap_df[context_features].iloc[0])
    
        y.append(lap_df["LapTime"].iloc[0])

    except Exception as e:
        print(f"Skipping lap: {lap_key} as error: {e}")

Processing Laps: 100%|██████████| 1770/1770 [00:03<00:00, 447.70it/s]


In [16]:
X_seq = np.stack(X_seq) 

In [17]:
X_seq.shape

(1770, 100, 6)

In [18]:
X_context = pd.DataFrame(X_context)

In [19]:
X_context.shape

(1770, 9)

In [20]:
y

['0 days 00:01:14.892000',
 '0 days 00:01:44.769000',
 '0 days 00:01:23.886000',
 '0 days 00:01:49.103000',
 '0 days 00:01:17.131000',
 '0 days 00:01:31.663000',
 '0 days 00:01:15.600000',
 '0 days 00:01:35.450000',
 '0 days 00:01:13.791000',
 '0 days 00:01:33.471000',
 '0 days 00:01:18.706000',
 '0 days 00:01:35.800000',
 '0 days 00:01:17.993000',
 '0 days 00:01:43.801000',
 '0 days 00:01:23.473000',
 '0 days 00:01:48.104000',
 '0 days 00:01:17.068000',
 '0 days 00:01:31.759000',
 '0 days 00:01:19.894000',
 '0 days 00:01:15.948000',
 '0 days 00:01:31.225000',
 '0 days 00:01:17.927000',
 '0 days 00:01:34.246000',
 '0 days 00:01:12.939000',
 '0 days 00:01:42.129000',
 '0 days 00:01:22.820000',
 '0 days 00:01:36.286000',
 '0 days 00:01:15.931000',
 '0 days 00:01:32.126000',
 '0 days 00:01:14.353000',
 '0 days 00:01:33.011000',
 '0 days 00:01:13.043000',
 '0 days 00:01:32.705000',
 '0 days 00:01:20.569000',
 '0 days 00:01:30.559000',
 '0 days 00:01:16.157000',
 '0 days 00:01:29.639000',
 

In [21]:
y_seconds = []

In [22]:
for val in y:
    try:
        lap_time = pd.to_timedelta(val)
        seconds = lap_time.total_seconds()
        y_seconds.append(seconds)

    except Exception as e:
        print(f"Skipping {val} cause Error {e}")

In [23]:
y = np.array(y_seconds, dtype = np.float32)

In [24]:
y

array([ 74.892, 104.769,  83.886, ...,  75.641,  75.543,  76.248],
      shape=(1770,), dtype=float32)

In [25]:
X_context.head()

,Driver,LapNumber,Year,Round,Compound,TyreLife,FreshTyre,Team,TrackStatus
99974,LEC,2.0,2022,monaco,SOFT,2.0,True,Ferrari,1
0,LEC,2.0,2022,silverstone,INTERMEDIATE,2.0,True,Ferrari,1
132522,LEC,2.0,2022,spa,HARD,2.0,True,Ferrari,1
198854,LEC,2.0,2022,suzuka,WET,2.0,True,Ferrari,1
331611,LEC,2.0,2023,monaco,HARD,2.0,True,Ferrari,1


In [26]:
X_context = pd.get_dummies(X_context, columns=["Compound", "TrackStatus","Team", "Driver", "Round"])

In [27]:
X_context.head()

,LapNumber,Year,TyreLife,FreshTyre,Compound_HARD,Compound_INTERMEDIATE,Compound_MEDIUM,Compound_SOFT,Compound_TEST_UNKNOWN,Compound_WET,...,TrackStatus_21,Team_Ferrari,Team_Red Bull Racing,Driver_LEC,Driver_SAI,Driver_VER,Round_monaco,Round_silverstone,Round_spa,Round_suzuka
99974,2.0,2022,2.0,True,False,False,False,True,False,False,...,False,True,False,True,False,False,True,False,False,False
0,2.0,2022,2.0,True,False,True,False,False,False,False,...,False,True,False,True,False,False,False,True,False,False
132522,2.0,2022,2.0,True,True,False,False,False,False,False,...,False,True,False,True,False,False,False,False,True,False
198854,2.0,2022,2.0,True,False,False,False,False,False,True,...,False,True,False,True,False,False,False,False,False,True
331611,2.0,2023,2.0,True,True,False,False,False,False,False,...,False,True,False,True,False,False,True,False,False,False


In [28]:
cols = X_context.columns

In [29]:
X_context["FreshTyre"].dtype

dtype('bool')

In [30]:
for col in tqdm(cols, desc="Convert Boolean to Int"):
    if X_context[col].dtype == bool:
        X_context[col] = X_context[col].astype(int)

Convert Boolean to Int: 100%|██████████| 22/22 [00:00<00:00, 4466.35it/s]


In [31]:
X_context.head()

,LapNumber,Year,TyreLife,FreshTyre,Compound_HARD,Compound_INTERMEDIATE,Compound_MEDIUM,Compound_SOFT,Compound_TEST_UNKNOWN,Compound_WET,...,TrackStatus_21,Team_Ferrari,Team_Red Bull Racing,Driver_LEC,Driver_SAI,Driver_VER,Round_monaco,Round_silverstone,Round_spa,Round_suzuka
99974,2.0,2022,2.0,1,0,0,0,1,0,0,...,0,1,0,1,0,0,1,0,0,0
0,2.0,2022,2.0,1,0,1,0,0,0,0,...,0,1,0,1,0,0,0,1,0,0
132522,2.0,2022,2.0,1,1,0,0,0,0,0,...,0,1,0,1,0,0,0,0,1,0
198854,2.0,2022,2.0,1,0,0,0,0,0,1,...,0,1,0,1,0,0,0,0,0,1
331611,2.0,2023,2.0,1,1,0,0,0,0,0,...,0,1,0,1,0,0,1,0,0,0


In [32]:
X_context = np.array(X_context)

In [33]:
X_context

array([[2.000e+00, 2.022e+03, 2.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.000e+00, 2.022e+03, 2.000e+00, ..., 1.000e+00, 0.000e+00,
        0.000e+00],
       [2.000e+00, 2.022e+03, 2.000e+00, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       ...,
       [7.600e+01, 2.024e+03, 2.500e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [7.700e+01, 2.024e+03, 2.600e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [7.800e+01, 2.024e+03, 2.700e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00]], shape=(1770, 22))

In [34]:
np.save("X_seq.npy", X_seq)
np.save("X_context.npy", X_context)
np.save("y", y)